# LazyGraphRAG in LangChain

## Introduction

In [LazyGraphRAG](https://www.microsoft.com/en-us/research/blog/lazygraphrag-setting-a-new-standard-for-quality-and-cost/), Microsoft demonstrates significant cost and performance benefits to delaying the construction of a knowledge graph.
This is largely because not all documents need to be analyzed.
However, it is also benefical that documents by the time documents are analyzed the question is already known, allowing irrelevant information to be ignored. 

We've noticed similar cost benefits to building a document graph linking content based on simple properties such as extracted keywords compared to building a complete knowledge graph.
For the Wikipedia dataset used in this notebook, we estimated it would have taken $70k to build a knowledege graph using the [example from LangChain](https://python.langchain.com/docs/how_to/graph_constructing/#llm-graph-transformer), while the document graph was basically free.

In this notebook we demonstrate how to populate a document graph with Wikipedia articles linked based on mentions in the articles and extracted keywords.
Keyword extraction uses a local [KeyBERT](https://maartengr.github.io/KeyBERT/) model, making it fast and cost-effective to construct these graphs.
We'll then show how to build out a chain which does the steps of Lazy GraphRAG -- retrieving articles, extracting claims from each community, ranking and selecting the top claims, and generating an answer based on those claims.

## Environment Setup

The following block will configure the environment from the Colab Secrets.
To run it, you should have the following Colab Secrets defined and accessible to this notebook:

- `OPENAI_API_KEY`: The OpenAI key.
- `ASTRA_DB_API_ENDPOINT`: The Astra DB API endpoint.
- `ASTRA_DB_APPLICATION_TOKEN`: The Astra DB Application token.
- `LANGCHAIN_API_KEY`: Optional. If defined, will enable LangSmith tracing.
- `ASTRA_DB_KEYSPACE`: Optional. If defined, will specify the Astra DB keyspace. If not defined, will use the default.

In [2]:
# Install modules.
#
# On Apple hardware, "spacy[apple]" will improve performance.
%pip install \
    langchain-core \
    langchain-astradb \
    langchain-openai \
    langchain-graph-retriever \
    spacy \
    graph-rag-example-helpers

  Using cached numpy-1.26.4-cp311-cp311-macosx_10_9_x86_64.whl.metadata (61 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
INFO: pip is looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
  Using cached h2-4.2.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached dnspython-2.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached hyperframe-6.1.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached hpack-4.1.0-py3-none-any.whl.metadata (4.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 7.7 MB/s eta 0:00:00a 0:00:01
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 5.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 23.4 MB/s eta 0:00:0000:0100:01
Using cached python_

The last package -- `graph-rag-example-helpers` -- includes some helpers for setting up environment helpers and allowing the loading of wikipedia data to be restarted if it fails.

spaCy는 자연어 처리(NLP, Natural Language Processing)를 위한 오픈소스 라이브러리입니다. 주요 기능으로는:
토큰화(Tokenization)
품사 태깅(Part-of-speech tagging)
개체명 인식(Named Entity Recognition, NER)
구문 분석(Dependency parsing)
문장 분할(Sentence segmentation)
등을 제공합니다.
!python -m spacy download en_core_web_sm 명령은 spaCy의 영어 언어 모델 중 하나인 'en_core_web_sm'을 다운로드하는 명령입니다.
en: 영어 모델
core: 기본 기능을 포함
web: 웹 텍스트에 최적화
sm: small 모델 (가벼운 버전)

In [3]:
# Downloads the model used by Spacy for extracting entities.
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 14.9 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
# Configure import paths.
import os
import sys

sys.path.append("../../")

# Initialize environment variables.
from graph_rag_example_helpers.env import Environment, initialize_environment

initialize_environment(Environment.ASTRAPY)

os.environ["LANGCHAIN_PROJECT"] = "lazy-graph-rag"

# The full dataset is ~6m documents, and takes hours to load.
# The short dataset is 1000 documents and loads quickly.
# Change this to `True` to use the larger dataset.
USE_SHORT_DATASET = True

## Part 1: Loading Data

# 먼저, `AstraDBVectorStore`에 위키백과 데이터를 로드하는 방법을 보여드리겠습니다. 이를 위해 언급된 기사와 키워드를 메타데이터 필드로 사용합니다.
# 이 섹션에서는 실제로 그래프에 특별한 작업을 수행하지 않습니다. 우리는 단지 우리의 내용을 유용하게 설명하는 필드를 메타데이터에 채워넣는 것입니다.

### Create Documents from Wikipedia Articles
# 먼저, 가져올 `LangChain` `Document`s를 생성해야 합니다.
# 
# 이를 위해, [2wikimultihop](https://github.com/Alab-NII/2wikimultihop?tab=readme-ov-file#new-update-april-7-2021)에서 다운로드한 JSON 파일의 줄을 변환하는 코드를 작성합니다. 이를 통해 `Document`를 생성합니다.
# 이 파일에 있는 정보를 사용하여 `id`와 `metadata["mentions"]`를 채워넣습니다.
# 
# 다음으로, `SpacyNERTransformer`를 통해 문서를 실행하여 기사에서 언급된 엔티티를 `metadata["entities"]`에 채워넣습니다.

In [5]:
import json
from collections.abc import Iterator

from langchain_core.documents import Document
from langchain_graph_retriever.transformers.spacy import (
    SpacyNERTransformer,
)


def parse_document(line: bytes) -> Document:
    """Reads one JSON line from the wikimultihop dump."""
    para = json.loads(line)

    id = para["id"]
    title = para["title"]

    # Use structured information (mentioned Wikipedia IDs) as metadata.
    mentioned_ids = [id for m in para["mentions"] for m in m["ref_ids"] or []]

    return Document(
        id=id,
        page_content=" ".join(para["sentences"]),
        metadata={
            "mentions": mentioned_ids,
            "title": title,
        },
    )


NER_TRANSFORMER = SpacyNERTransformer(
    limit=1000,
    exclude_labels={"CARDINAL", "MONEY", "QUANTITY", "TIME", "PERCENT", "ORDINAL"},
)


# Load data in batches, using GLiNER to extract entities.
def prepare_batch(lines: Iterator[str]) -> Iterator[Document]:
    # Parse documents from the batch of lines.
    docs = [parse_document(line) for line in lines]

    docs = NER_TRANSFORMER.transform_documents(docs)

    return docs

### Create the AstraDBVectorStore
Next, we create the Vector Store we're going to load these documents into.
In our case, we use DataStax Astra DB with Open AI embeddings.

In [6]:
from langchain_astradb import AstraDBVectorStore
from langchain_openai import OpenAIEmbeddings

COLLECTION = "lazy_graph_rag_short" if USE_SHORT_DATASET else "lazy_graph_rag"
store = AstraDBVectorStore(
    embedding=OpenAIEmbeddings(),
    collection_name=COLLECTION,
    pre_delete_collection=USE_SHORT_DATASET,
)

### Loading Data into the Store
Next, we perform the actual loading.
This takes a while, so we use a helper utility to persist which batches have been written so we can resume if there are any failures.

On OS X, it is useful to run `caffeinate -dis` in a shell to prevent the machine from going to sleep and seems to reduce errors.

이 코드는 Wikipedia 데이터를 로드하고 처리하는 명령입니다. 구체적으로 살펴보면:

```python
await aload_2wikimultihop(
    limit=100 if USE_SHORT_DATASET else None,
    full_para_with_hyperlink_zip_path=PARA_WITH_HYPERLINK_ZIP,
    store=store,
    batch_prepare=prepare_batch,
)
```

1. `aload_2wikimultihop`: 2wikimultihop 데이터셋을 비동기적으로 로드하는 함수입니다. 2wikimultihop은 Wikipedia 문서들의 데이터셋으로, 문서들 간의 하이퍼링크 관계 정보를 포함하고 있습니다.

2. 주요 매개변수:
   - `limit=100 if USE_SHORT_DATASET else None`: 
     - `USE_SHORT_DATASET`가 True이면 100개의 문서만 로드
     - False이면 전체 데이터셋(약 6백만 문서) 로드
   
   - `full_para_with_hyperlink_zip_path=PARA_WITH_HYPERLINK_ZIP`:
     - Wikipedia 데이터가 포함된 ZIP 파일의 경로

   - `store=store`: 
     - 처리된 데이터를 저장할 AstraDB 벡터 저장소

   - `batch_prepare=prepare_batch`:
     - 문서를 처리하는 함수로, 각 문서에서 엔티티를 추출하고 메타데이터를 준비

이 함수는 다음과 같은 작업을 수행합니다:
1. Wikipedia 문서들을 ZIP 파일에서 읽어옴
2. 각 문서에서 관련 정보(제목, 본문, 하이퍼링크 등) 추출
3. SpaCy를 사용하여 각 문서에서 엔티티(사람, 장소, 조직 등) 추출
4. 처리된 데이터를 AstraDB 벡터 저장소에 저장

이렇게 저장된 데이터는 나중에 LazyGraphRAG 시스템에서 문서 간의 관계를 분석하고 질문에 답변하는 데 사용됩니다.


In [7]:
import os
import os.path

from graph_rag_example_helpers.datasets.wikimultihop import aload_2wikimultihop

# Path to the file `para_with_hyperlink.zip`.
# See instructions here to download from
# [2wikimultihop](https://github.com/Alab-NII/2wikimultihop?tab=readme-ov-file#new-update-april-7-2021).
PARA_WITH_HYPERLINK_ZIP = os.path.join(os.getcwd(), "para_with_hyperlink.zip")

await aload_2wikimultihop(
    limit=100 if USE_SHORT_DATASET else None,
    full_para_with_hyperlink_zip_path=PARA_WITH_HYPERLINK_ZIP,
    store=store,
    batch_prepare=prepare_batch,
)

Loaded from ../../data/para_with_hyperlink_short.jsonl


At this point, we've created a `VectorStore` with the Wikipedia articles.
Each article is associated with metadata identifying other articles it mentions and entities from the article.

As is, this is useful for performing a vector search filtered to articles mentioning a specific term or performing an entity seach on the documents.
The library `langchain-graph-retriever` makes this even more useful by allowing articles to be traversed based on relationships such as articles mentioned in the current article (or mentioning the current article) or articles providing more information on the entities mentioned in the current article.

In the next section we'll see not just how we can use the relationships in the metadata to retrieve more articles, but we'll go a step further and perform Lazy GraphRAG to extract relevant claims from both the similar and related articles and use the most relevant claims to answer the question.

## Part 2: Lazy Graph RAG via Hierarchical Summarization

As we've noted before, eagerly building a knowledge graph is prohibitively expensive.
Microsoft seems to agree, and recently introduced LazyGraphRAG, which enables GraphRAG to be performed late -- after a query is retrieved.

We implement the LazyGraphRAG technique using the traversing retrievers as follows:

1. Retrieve a good number of nodes using a traversing retrieval.
2. Identify communities in the retrieved sub-graph.
3. Extract claims from each community relevant to the query using an LLM.
4. Rank each of the claims based on the relevance to the question and select the top claims.
5. Generate an answer to the question based on the extracted claims.

이전에 언급했듯이, 지식 그래프를 적극적으로 구축하는 것은 엄청나게 비용이 듭니다.
마이크로소프트도 동의하며, 최근에 LazyGraphRAG를 도입했습니다. 이는 쿼리가 검색된 후에 GraphRAG를 수행할 수 있게 합니다.
우리는 다음과 같은 방법으로 트래버싱 리트리버를 사용하여 LazyGraphRAG 기법을 구현합니다:

1. 트래버싱 리트리벌을 사용하여 적절한 수의 노드를 검색합니다.
2. 검색된 하위 그래프에서 커뮤니티를 식별합니다.
3. LLM을 사용하여 쿼리와 관련된 각 커뮤니티에서 주장을 추출합니다.
4. 질문에 대한 관련성을 기준으로 각 주장을 순위付け하고, 최상위 주장을 선택합니다.
5. 추출된 주장에 기반하여 질문에 대한 답을 생성합니다.

### LangChain for Extracting Claims

The first thing we do is create a chain that produces the claims. Given an input containing the question and the retrieved communities, it applies an LLM in parallel extracting claims from each community.

A claim is just a string representing the statement and the `source_id` of the document. We request structured output so we get a list of claims.

첫 번째로, 주장을 생성하는 체인을 만듭니다. 질문과 검색된 커뮤니티를 포함하는 입력이 주어지면, 각 커뮤니티에서 주장을 병렬로 추출하는 LLM을 적용합니다.
 
주장은 문서의 `source_id`와 문장으로 구성된 단순한 문자열입니다. 구조화된 출력을 요청하여 주장의 목록을 얻습니다.

In [8]:
from collections.abc import Iterable
from operator import itemgetter
from typing import TypedDict

from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableParallel, chain
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


class Claim(BaseModel):
    """Representation of an individual claim from a source document(s)."""

    claim: str = Field(description="The claim from the original document(s).")
    source_id: str = Field(description="Document ID containing the claim.")


class Claims(BaseModel):
    """Claims extracted from a set of source document(s)."""

    claims: list[Claim] = Field(description="The extracted claims.")


MODEL = ChatOpenAI(model="gpt-4o", temperature=0)
CLAIMS_MODEL = MODEL.with_structured_output(Claims)

CLAIMS_PROMPT = ChatPromptTemplate.from_template("""
Extract claims from the following related documents.

Only return claims appearing within the specified documents.
If no documents are provided, do not make up claims or documents.

Claims (and scores) should be relevant to the question.
Don't include claims from the documents if they are not directly or indirectly
relevant to the question.

If none of the documents make any claims relevant to the question, return an
empty list of claims.

If multiple documents make similar claims, include the original text of each as
separate claims. Score the most useful and authoritative claim higher than
similar, lower-quality claims.

Question: {question}

{formatted_documents}
""")

# TODO: Few-shot examples? Possibly with a selector?


def format_documents_with_ids(documents: Iterable[Document]) -> str:
    formatted_docs = "\n\n".join(
        f"Document ID: {doc.id}\nContent: {doc.page_content}" for doc in documents
    )
    return formatted_docs


CLAIM_CHAIN = (
    RunnableParallel(
        {
            "question": itemgetter("question"),
            "formatted_documents": itemgetter("documents")
            | RunnableLambda(format_documents_with_ids),
        }
    )
    | CLAIMS_PROMPT
    | CLAIMS_MODEL
)


class ClaimsChainInput(TypedDict):
    question: str
    communities: Iterable[Iterable[Document]]


@chain
async def claims_chain(input: ClaimsChainInput) -> Iterable[Claim]:
    question = input["question"]
    communities = input["communities"]

    # TODO: Use openai directly so this can use the batch API for performance/cost?
    community_claims = await CLAIM_CHAIN.abatch(
        [{"question": question, "documents": community} for community in communities]
    )
    return [claim for community in community_claims for claim in community.claims]

`RunnableParallel`은 여기서 두 가지 작업을 병렬로 처리합니다:

1. `"question": itemgetter("question")`:
   - 입력에서 질문(question)을 추출하는 작업

2. `"formatted_documents": itemgetter("documents") | RunnableLambda(format_documents_with_ids)`:
   - 입력에서 문서(documents)를 추출하고
   - 추출된 문서들을 포맷팅하는 작업 (`format_documents_with_ids` 함수를 통해)

이 두 작업이 병렬로 실행되어 다음과 같은 구조의 딕셔너리를 생성합니다:
```python
{
    "question": "추출된 질문",
    "formatted_documents": "ID와 함께 포맷팅된 문서들"
}
```

이렇게 생성된 딕셔너리는 파이프라인(`|`)을 통해:
1. `CLAIMS_PROMPT`로 전달되어 프롬프트 템플릿이 채워지고
2. 최종적으로 `CLAIMS_MODEL`에 의해 처리되어 관련 주장(claims)들이 추출됩니다.

이러한 병렬 처리는 성능 최적화를 위한 것으로, 질문 추출과 문서 포맷팅이 서로 독립적인 작업이므로 동시에 처리할 수 있게 합니다.

`itemgetter`는 Python의 `operator` 모듈에서 제공하는 함수로, 주어진 키나 인덱스를 사용하여 객체에서 항목을 추출하는 콜러블(callable) 객체를 생성합니다.

예를 들어 설명하면:

```python
from operator import itemgetter

# 딕셔너리의 경우
data = {"question": "What is Python?", "documents": ["doc1", "doc2"]}
get_question = itemgetter("question")
result = get_question(data)  # "What is Python?"

# 리스트의 경우
data = ["a", "b", "c"]
get_first = itemgetter(0)
result = get_first(data)  # "a"
```

코드에서 사용된 예시를 보면:
```python
"question": itemgetter("question")
```
이 부분은:
1. 입력 데이터에서 "question" 키를 가진 값을 추출하는 함수를 생성
2. 예를 들어 입력이 `{"question": "Why is sky blue?", "documents": [...]}` 라면
3. `"Why is sky blue?"` 값을 추출

```python
"formatted_documents": itemgetter("documents")
```
이 부분은:
1. 입력 데이터에서 "documents" 키를 가진 값을 추출하는 함수를 생성
2. 추출된 문서들은 이후 `format_documents_with_ids` 함수를 통해 포맷팅됨

`itemgetter`의 장점:
1. 간결한 문법: `lambda x: x["question"]` 대신 `itemgetter("question")`처럼 간단하게 표현
2. 성능: `lambda` 함수보다 더 효율적
3. 여러 키/인덱스 동시 추출 가능: `itemgetter("a", "b")(data)` 처럼 사용 가능

이 코드에서는 입력 데이터에서 필요한 부분(질문과 문서)을 효율적으로 추출하기 위해 `itemgetter`를 사용하고 있습니다.

네, 정확히 맞습니다. `CLAIM_CHAIN`이 실제로 실행되는 부분입니다. 코드를 자세히 분석해보면:

```python
community_claims = await CLAIM_CHAIN.abatch(
    [{"question": question, "documents": community} for community in communities]
)
```

1. `communities`는 문서 그룹들의 리스트입니다.

2. 리스트 컴프리헨션으로 각 커뮤니티마다 다음과 같은 딕셔너리를 생성합니다:
   ```python
   {
       "question": question,      # 동일한 질문
       "documents": community     # 각각 다른 문서 그룹
   }
   ```

3. `.abatch()`는 이러한 입력들을 병렬로 처리합니다. 각 입력에 대해:
   - `itemgetter`로 question과 documents를 추출
   - documents는 `format_documents_with_ids`로 포맷팅
   - 포맷팅된 결과로 `CLAIMS_PROMPT` 생성
   - `CLAIMS_MODEL`로 관련 주장들을 추출

4. 비동기(`await`)로 처리되어 성능을 최적화합니다.

예를 들어, 3개의 커뮤니티가 있다면:
```python
[
    {"question": "Why is sky blue?", "documents": community1},
    {"question": "Why is sky blue?", "documents": community2},
    {"question": "Why is sky blue?", "documents": community3}
]
```
이런 형태의 입력들이 병렬로 처리되어 각 커뮤니티에서 관련된 주장들을 추출하게 됩니다.



### LangChain for Ranking Claims

The next chain is used for ranking the claims so we can select the most relevant to the question.

This is based on ideas from [RankRAG](https://arxiv.org/abs/2407.02485).
Specifically, the prompt is constructed so that the next token should be `True` if the content is relevant and `False` if not.
The probability of the token is used to determine the relevance -- `True` with a higher probability is more relevant than `True` with a lesser probability.

In [9]:
import math

from langchain_core.runnables import chain

RANK_PROMPT = ChatPromptTemplate.from_template("""
Rank the relevance of the following claim to the question.
Output "True" if the claim is relevant and "False" if it is not.
Only output True or False.

Question: Where is Seattle?

Claim: Seattle is in Washington State.

Relevant: True

Question: Where is LA?

Claim: New York City is in New York State.

Relevant: False

Question: {question}

Claim: {claim}

Relevant:
""")


def compute_rank(msg):
    logprob = msg.response_metadata["logprobs"]["content"][0]
    prob = math.exp(logprob["logprob"])
    token = logprob["token"]
    if token == "True":
        return prob
    elif token == "False":
        return 1.0 - prob
    else:
        raise ValueError(f"Unexpected logprob: {logprob}")


RANK_CHAIN = RANK_PROMPT | MODEL.bind(logprobs=True) | RunnableLambda(compute_rank)


class RankChainInput(TypedDict):
    question: str
    claims: Iterable[Claim]


@chain
async def rank_chain(input: RankChainInput) -> Iterable[Claim]:
    # TODO: Use openai directly so this can use the batch API for performance/cost?
    claims = input["claims"]
    ranks = await RANK_CHAIN.abatch(
        [{"question": input["question"], "claim": claim} for claim in claims]
    )
    rank_claims = sorted(
        zip(ranks, claims, strict=True), key=lambda rank_claim: rank_claim[0]
    )

    return [claim for _, claim in rank_claims]

이 코드는 LazyGraphRAG 시스템에서 주장(claims)의 관련성을 평가하고 순위를 매기는 부분입니다. 자세히 설명해드리겠습니다:

1. **RANK_PROMPT (순위 매기기 프롬프트)**
```python
RANK_PROMPT = ChatPromptTemplate.from_template("""
...
""")
```
- 이 프롬프트는 주어진 질문에 대해 각 주장의 관련성을 평가하는 템플릿입니다
- 예시를 포함하여 LLM이 이해하기 쉽게 구성되어 있습니다:
  - "Seattle은 어디에 있나요?" 라는 질문에 "Seattle은 Washington State에 있다"는 주장은 관련이 있으므로 "True"
  - "LA는 어디에 있나요?" 라는 질문에 "New York City는 New York State에 있다"는 주장은 관련이 없으므로 "False"

2. **compute_rank (순위 계산 함수)**
```python
def compute_rank(msg):
    logprob = msg.response_metadata["logprobs"]["content"][0]
    prob = math.exp(logprob["logprob"])
    token = logprob["token"]
```
- LLM의 응답에서 확률값을 계산하는 함수입니다
- `logprob`(로그 확률)을 실제 확률값으로 변환합니다
- "True" 토큰이 나오면 그 확률을 반환하고, "False" 토큰이 나오면 (1 - 확률)을 반환합니다
- 이를 통해 주장의 관련성에 대한 수치적인 점수를 얻을 수 있습니다

3. **RANK_CHAIN (순위 매기기 체인)**
```python
RANK_CHAIN = RANK_PROMPT | MODEL.bind(logprobs=True) | RunnableLambda(compute_rank)
```
- 프롬프트, 모델, 순위 계산을 하나의 파이프라인으로 연결합니다
- `logprobs=True`로 설정하여 모델이 확률값을 반환하도록 합니다

4. **rank_chain (순위 매기기 비동기 함수)**
```python
@chain
async def rank_chain(input: RankChainInput) -> Iterable[Claim]:
    claims = input["claims"]
    ranks = await RANK_CHAIN.abatch([...])
    rank_claims = sorted(zip(ranks, claims, strict=True), key=lambda rank_claim: rank_claim[0])
```
- 여러 주장들을 배치로 처리하여 효율적으로 순위를 매깁니다
- 각 주장에 대한 관련성 점수를 계산하고, 이를 기준으로 정렬합니다
- 가장 관련성 높은 주장들부터 낮은 순으로 정렬된 리스트를 반환합니다

이 시스템은 RankRAG라는 논문의 아이디어를 기반으로 하며, LLM의 다음 토큰 예측 확률을 활용하여 주장의 관련성을 평가합니다. 이는 단순히 "관련있다/없다"의 이진 분류가 아닌, 확률에 기반한 더 세밀한 순위 매기기를 가능하게 합니다.

모든 LLM이 logprobs를 제공하지는 않습니다. 이에 대해 자세히 알아보겠습니다:

1. **OpenAI GPT 모델들**
- OpenAI의 API는 logprobs를 제공합니다
- 하지만 ChatGPT API (gpt-3.5-turbo, gpt-4 등)는 기본적으로 logprobs를 제공하지 않습니다
- 텍스트 완성 모델(text-davinci-003 등)에서는 logprobs 파라미터를 사용할 수 있습니다

2. **다른 상용 LLM 서비스들**
- Anthropic Claude: logprobs를 제공하지 않습니다
- Google PaLM/Gemini: logprobs를 제공하지 않습니다
- Cohere: logprobs를 제공합니다
- AI21: logprobs를 제공합니다

3. **오픈소스 모델들**
- 대부분의 오픈소스 모델들은 로컬에서 실행할 때 logprobs 계산이 가능합니다
- 하지만 이는 추가적인 계산 비용이 들고 성능에 영향을 미칠 수 있습니다

따라서 이 코드의 ranking 방식은 모든 LLM에서 사용할 수 있는 것은 아닙니다. logprobs를 제공하지 않는 모델을 사용할 경우 대안적인 ranking 방법을 사용해야 합니다:

1. **대안적인 ranking 방법들**:
```python
# 예시 1: 이진 분류 방식
def alternative_rank_1(response):
    return 1.0 if response.strip().lower() == "true" else 0.0

# 예시 2: 점수 기반 방식
SCORE_PROMPT = """
Rate the relevance of the claim to the question on a scale of 0-10:
Question: {question}
Claim: {claim}
Score (0-10):
"""
```

2. **임베딩 기반 ranking**:
```python
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

def compute_similarity_rank(question, claim):
    q_embedding = embeddings.embed_query(question)
    c_embedding = embeddings.embed_query(claim)
    return cosine_similarity(q_embedding, c_embedding)
```

3. **Cross-encoder 기반 ranking**:
```python
from sentence_transformers import CrossEncoder

model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

def compute_relevance_score(question, claim):
    score = model.predict([question, claim])
    return score
```

이러한 대안적인 방법들은 logprobs에 의존하지 않으면서도 효과적인 ranking을 수행할 수 있습니다. 실제 구현시에는 사용하는 LLM의 특성과 제약사항을 고려하여 적절한 ranking 방법을 선택해야 합니다.



We could extend this by using an MMR-like strategy for selecting claims.
Specifically, we could combine the relevance of the claim to the question and the diversity compared to already selected claims to select the best variety of claims.

### LazyGraphRAG in LangChain

Finally, we produce a chain that puts everything together.
Given a `GraphRetriever` it retrieves documents, creates communities using edges amongst the retrieved documents, extracts claims from those communities, ranks and selects the best claims, and then answers the question using those claims.

In [10]:
from typing import Any

from graph_retriever.edges import EdgeSpec, MetadataEdgeFunction
from langchain_core.language_models import BaseLanguageModel
from langchain_core.runnables import chain
from langchain_graph_retriever import GraphRetriever
from langchain_graph_retriever.document_graph import create_graph, group_by_community


@chain
async def lazy_graph_rag(
    question: str,
    *,
    retriever: GraphRetriever,
    model: BaseLanguageModel,
    edges: Iterable[EdgeSpec] | MetadataEdgeFunction | None = None,
    max_tokens: int = 1000,
    **kwargs: Any,
) -> str:
    """Retrieve claims relating to the question using LazyGraphRAG.

    Returns the top claims up to the given `max_tokens` as a markdown list.

    """
    edges = edges or retriever.edges
    if edges is None:
        raise ValueError("Must specify 'edges' in invocation or retriever")

    # 1. Retrieve documents using the (traversing) retriever.
    documents = await retriever.ainvoke(question, edges=edges, **kwargs)

    # 2. Create a graph and extract communities.
    document_graph = create_graph(documents, edges=edges)
    communities = group_by_community(document_graph)

    # 3. Extract claims from the communities.
    claims = await claims_chain.ainvoke(
        {"question": question, "communities": communities}
    )

    # 4. Rank the claims and select claims up to the given token limit.
    result_claims = []
    tokens = 0

    for claim in await rank_chain.ainvoke({"question": question, "claims": claims}):
        claim_str = f"- {claim.claim} (Source: {claim.source_id})"

        tokens += model.get_num_tokens(claim_str)
        if tokens > max_tokens:
            break
        result_claims.append(claim_str)

    return "\n".join(result_claims)

### Using Lazy GraphRAG in LangChain

Finally, we sue the Lazy GraphRAG chain we created on the store we populated earlier.

In [11]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_graph_retriever import GraphRetriever

RETRIEVER = GraphRetriever(
    store=store,
    edges=[("mentions", "$id"), ("entities", "entities")],
    k=100,
    start_k=30,
    adjacent_k=20,
    max_depth=3,
)

ANSWER_PROMPT = PromptTemplate.from_template("""
Answer the question based on the supporting claims.

Only use information from the claims. Do not guess or make up any information.

Where possible, reference and quote the supporting claims.

Question: {question}

Claims:
{claims}
""")

LAZY_GRAPH_RAG_CHAIN = (
    {
        "question": RunnablePassthrough(),
        "claims": RunnablePassthrough()
        | lazy_graph_rag.bind(
            retriever=RETRIEVER,
            model=MODEL,
            max_tokens=1000,
        ),
    }
    | ANSWER_PROMPT
    | MODEL
)

In [12]:
QUESTION = "Why are Bermudan sloop ships widely prized compared to other ships?"
result = await LAZY_GRAPH_RAG_CHAIN.ainvoke(QUESTION)
result.content

"I'm sorry, but it seems that the claims needed to answer the question are missing. Please provide the claims so I can help you with your question."

For comparison, below are the results to the same question using a basic RAG pattern with just vector similarity.

In [13]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

VECTOR_ANSWER_PROMPT = PromptTemplate.from_template("""
Answer the question based on the provided documents.

Only use information from the documents. Do not guess or make up any information.

Question: {question}

Documents:
{documents}
""")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


VECTOR_CHAIN = (
    {
        "question": RunnablePassthrough(),
        "documents": (store.as_retriever() | format_docs),
    }
    | VECTOR_ANSWER_PROMPT
    | MODEL
)

result = VECTOR_CHAIN.invoke(QUESTION)
result.content

'The provided documents do not contain any information about Bermudan sloop ships or why they might be widely prized compared to other ships.'

The LazyGraphRAG chain is great when a question needs to consider a large amount of relevant information in order to produce a thorough answer.

## Conclusion

This post demonstrated how easy it is to implement Lazy GraphRAG on top of a document graph.

It used `langchain-graph-retriever` from the [graph-rag project](datastax.github.io/graph-rag) to implement the document graph and graph-based retrieval on top of an existing LangChain `VectorStore`.
This means you can focus on populating and using your `VectorStore` with useful metadata and add graph-based retrieval and even Lazy GraphRAG when you need it.

**Any LangChain `VectorStore` can be used with Lazy GraphRAG without needing to change or re-ingest the stored documents.**
Knowledge Graphs and GraphRAG shouldn't be hard or scary.
Start simple and easily overlay edges when you need them.

Graph retrievers and LazyGraph RAG work well with agents.
You can allow the agent to retrieve differently depending on the question -- doing a vector only search for simple questions, traversing to mentioned articles for a deeper question or traversing to articles that cite this to see if there is newer information available.
We'll show how to combine these techniques with agents in a future post.
Until then, give `langchain-graph-retriever` a try and let us know how it goes!